![](https://velog.velcdn.com/images/hyejinbeck/post/e69505c7-b75a-410d-861a-73956babf1d1/image.png)

![](https://velog.velcdn.com/images/hyejinbeck/post/ed682230-205b-4cc7-a6c9-6f5e94010bc0/image.png)

![](https://velog.velcdn.com/images/hyejinbeck/post/da5ed408-2078-46d8-a093-97d00d6dd326/image.png)

In [6]:
# 초기설치 및 불러오기
# !pip install -r requirements.txt      하고나서 주석처리 
import pandas as pd 
data = pd.read_csv('./Dataset/Dataset_04.csv')
data.info()
data.head()

#################### 문제 1 

# 한국인 = 한국데이터만 추출 
data.LOCATION.unique()      # 한국은 'KOR'
data['LOCATION'].unique()   # 동일한 기능 

q1 = data[data.LOCATION == 'KOR']
q1.info()

# 년도별 육류 소비량 합계 구하기 (pivot이용하거나 crosstab이용하거나)
pd.pivot_table(q1, index = 'TIME', values = 'Value', aggfunc = 'sum')
pd.pivot_table(q1, index = 'TIME', values = 'Value', aggfunc = 'sum').reset_index()
q1_tab = pd.pivot_table(q1, index = 'TIME', values = 'Value', aggfunc = 'sum').reset_index()

# 그냥 참고 = crosstab 이용할경우 
pd.crosstab(index=q1['TIME'], columns='Value', values=q1['Value'], aggfunc='sum').reset_index()

# TIME과 Value간의 상관분석 (value가 한국인의 년도별 육류 소비량 합계) 
q1_tab.corr().loc['TIME','Value']

# 상관계수를 반올림하여, 소수점 둘째자리까지 구하기 
round(q1_tab.corr().loc['TIME','Value'], 3)

#################### 문제 2 

# 한국 KOR, 일본 JPN 데이터 추출 
q2 = data[data.LOCATION.isin(['KOR','JPN'])]
q2

# 육류별 = 육류 종류 확인 
q2.SUBJECT.unique()    # array(['BEEF', 'PIG', 'POULTRY', 'SHEEP'], dtype=object)

# 육류별 소비량을 두 국가별로 연도기준 
q2_tab = pd.pivot_table(q2, index=['TIME','SUBJECT'], columns = 'LOCATION')
q2_tab
q2_tab = pd.pivot_table(q2, index=['TIME','SUBJECT'], columns = 'LOCATION', values='Value')
q2_tab

# 대응표본 t 검정 준비 = 반드시 데이터의 갯수 동일 = 들어가는 순서 동일해야함 
q2_tab = q2_tab.dropna()    # 결측치 제거 = 그래야 쌍이 맞음 

# 대응표본 t 검정 함수
from scipy.stats import ttest_rel

# 대응표본 t 검정 진행 
ttest_rel(q2_tab['KOR'], q2_tab['JPN'])
q2_out = ttest_rel(q2_tab['KOR'], q2_tab['JPN'])
print(q2_out)
q2_out.statistic
print(f'{q2_out.statistic:.3f}')

#################### 문제 3 

# 한국만 포함한 데이터만 추출 
q3 = data[data.LOCATION == 'KOR']

# 육류 종류(그룹)별 회귀분석할 함수
from sklearn.linear_model import LinearRegression

# ######################################## sample 
# # TIME을 독립변수x로, Value를 종속변수y로하여 육류종류 SUBJECT 별로 회귀분석 설계 = 결정계수 추출 
# temp_s = q3[q3.SUBJECT == 'BEEF']  # 일단 육류종류 SUBJECT가 BEEF인것만 
# temp_s

# # model_error = LinearRegression().fit(temp['TIME'], temp['Value']) 
# # error 해석 : using array.reshape(-1, 1)

# # 회귀분석 모델 진행 
# # 차원을 바꿔야함 
# # temp[['TIME']]
# model_s = LinearRegression().fit(temp_s[['TIME']], temp_s['Value']) 

# # 회귀분석 결과값 = 결정계수값 확인 
# model_s.score(temp_s[['TIME']], temp_s['Value'])  # 0.835455805530312 

# # 육류 종류(그룹)별 회귀계수와 예측값 구하기 
# pred_s = model_s.predict(temp_s[['TIME']])

# # 다시 회귀분석 결과값 
# r2_s = model_s.score(temp_s[['TIME']], temp_s['Value']) 
# print(r2_s)

# # 가장 높은 결정계수를 가진 모델의 학습오차 중 MAPE 구하기 
# (temp_s['Value']-pred_s) / temp_s['Value']
# # temp['Value'] 는 양수 혹은 음수가 될 수 있어서 절대값을 취해야함. 
# ( abs(temp_s['Value'] - pred_s) / temp_s['Value'])
# ( abs(temp_s['Value'] - pred_s) / temp_s['Value']).sum()
# ( abs(temp_s['Value'] - pred_s) / temp_s['Value']).sum() * 100 
# ( abs(temp_s['Value'] - pred_s) / temp_s['Value']).sum() * 100  / len(temp_s)
# mape_s = ( abs(temp_s['Value'] - pred_s) / temp_s['Value']).sum() * 100  / len(temp_s)

######################################## 

# 근데 이제 육류 종류(그룹)별 구해야되서 다시 

sub_list = q3.SUBJECT.unique()

model = LinearRegression().fit(temp[['TIME']], temp['Value']) 


q3_out = []
for i in sub_list : 
    temp = q3[q3.SUBJECT == i ]       # i 는 육류 종류별 
    model.score(temp[['TIME']], temp['Value'])
    pred = model.predict(temp[['TIME']])
    r2 = model.score(temp[['TIME']], temp['Value']) 
    mape = ( abs(temp['Value'] - pred) / temp['Value']).sum() * 100  / len(temp)
    q3_out.append([i,r2,mape])
print(q3_out)
q3_out

# MAPE 학습 오차 (성능평가지표) 
q3_out = pd.DataFrame(q3_out, columns = ['sub','r2','mape']) # append 값 들어갔던 순서대로 columns 명 지정 
q3_out.sort_values
q3_out.sort_values('r2', ascending=False).head(1)  # r2기준으로 내림차순해서 가장 맨 위에 있는 값 = MAPE 가장 큰 값 

q3_answer = q3_out.sort_values('r2', ascending=False).head(1)
q3_answer

# 반올림하여 소수점 둘쨰자리까지 
round(q3_answer, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6718 entries, 0 to 6717
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   LOCATION  6718 non-null   object 
 1   SUBJECT   6718 non-null   object 
 2   TIME      6718 non-null   int64  
 3   Value     6718 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 210.1+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, 442 to 587
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   LOCATION  146 non-null    object 
 1   SUBJECT   146 non-null    object 
 2   TIME      146 non-null    int64  
 3   Value     146 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 5.7+ KB
TtestResult(statistic=5.662768873304346, pvalue=7.731889627475099e-08, df=145)
5.663
[['BEEF', -22.123008265285986, 97.93612798008533], ['PIG', -13.801405280165593, 99.19188261824794], ['POULTRY', -1

,sub,r2,mape
3,SHEEP,0.004,33.348


# 강의 정답과 다르다! 
마지막 정답 q3_out.sort_values('r2', ascending=False).head(1) 하게되면 <br>
강의에서는 POULTRY(sub), 0.9514(r2), 5.783(mape)가 나온다. <br>
내가 잘못 작성한 부분이 있는 것 같다. <br> 
다시 강의를 반복들으며, 빠지거나 오입력된 부분이 있는지 찾아보겠다. 